<center>

# **Introduction to Machine Learning Project CLC-24-2**

<center>

## **Student Information**

<center>

| **Student Name** | **Student ID** |
| --- | --- |
| Đặng Thanh Tú | 22127432 |
| Nguyễn Công Tuấn | 22127436 |

</center>

## **Import modules/packages**

$pip install numpy pandas matplotlib seaborn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

## **Exploratory Data Analysis (EDA)**

This step concerns the analysis of obtained data to *understand its characteristics* and to *identify potential issues that may affect modeling*. There are several tasks related to this step, including but not limit to:

### **Data Collection**

Gather data either to have a preliminary dataset for the first time or to supplement the data from previous phases(s).

In [5]:
# We will handle later

### **Data Preprocessing**

Clean the data by *handling missing values*, *normalizing* or *standardizing features* and *encoding categorical variables* as needed.

**Read the dataset**

In [14]:
train = pd.read_csv('../data/train.csv')

### **Feature Engineering**

*Extract* or *construct* new features as you need to *augment the newly obtained data* or to *mitigate problems from prior phase(s)*.

## **Model Development**

This is when you develops models to *capture the patterns revealed during the analysis*. This step involves *selecting appropriate modeling approach* and *design a detailed algorithm*.

### **Model Selection**

Research to find an appropriate modeling algorithm that may help you *capture the patterns within this data* as well as to *define relevant information about the model* including its hyper-parameters and training procedure.

### **Training**

*Train the selected algorithm* with chosen approach on the prepared dataset to obtain a final model.

## **Model Evaluation**

You will then *evaluate that model's performance* using *metrics* relevant to the problem as well as desired result of the project and *perform analysis* on the trained model's limitations to *identify rooms for improvement*.

### **Evaluation**

*Assess the model* using *metrics* such as mean squared error (MSE), mean percentage error (MPE) or root-mean-squared logarithmic error (RMSLE) for *regression problems* and accuracy, precision, recall or AUC-ROC for *classification problems*.

### **Limitation Analysis**

*Analysis the erroneous cases* that are incorrectly predicted by the model which are reflected by the above metrics and *identify their properties* as well as *reasons for their inaccuracy*